In [2]:
import numpy as np
from py_pcha.PCHA import PCHA
import pandas as pd

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn import mixture
from sklearn.cluster import KMeans

In [5]:
def my_scaler(x):
    """standardize the predictors"""
    
    new_x = pd.DataFrame(scale(x, axis=0, with_mean=True, with_std=True, copy=True))
    new_x.columns = x.columns
    
    return new_x

def my_normal(x):
    """normalize the predictors"""
    
    scaler = MinMaxScaler() 
    scaled_values = scaler.fit_transform(x)
    new_x = pd.DataFrame(scaled_values)
    new_x.columns = x.columns

    return new_x

In [6]:
final_cluster_clean_id=pd.read_csv("final_cluster_clean_5_10.csv")
final_cluster_clean=final_cluster_clean_id.drop(["playersID","teamID"],axis=1)

In [9]:
def cluster(dt,cluster_content,scale, cluster_number, cluster_method):
    player_feature_df=dt
    performance_cluster=[]
    weapon_cluster=[]
    
    if scale=="MinMax":
        min_max_scaler = preprocessing.MinMaxScaler()
        player_feature_df = my_normal(player_feature_df)
        
    if scale=="Standardization":
        player_feature_df = my_scaler(player_feature_df)
            
    if cluster_content=="performance":
        cluster_dt=player_feature_df[['Ability', 'Grenade', 'Melee','Super', 'assists']]
        
    if cluster_content=="weapon":
        cluster_dt=player_feature_df[['Pro Auto Rifle',
                                      'Pro Fusion Rifle', 'Pro Grenade Launcher', 'Pro Hand Cannon',
                                      'Pro Linear Fusion Rifle', 'Pro Pulse Rifle', 'Pro Rocket Launcher',
                                      'Pro Scout Rifle', 'Pro Shotgun', 'Pro Sidearm', 'Pro Sniper Rifle',
                                      'Pro Submachine Gun', 'Pro Sword', 'Pro Trace Rifle']]   
        
    if cluster_method=="GMM":
        clf = mixture.GaussianMixture(n_components=cluster_number, covariance_type='full')
        clf.fit(cluster_dt)
        cluster_dt = cluster_dt.copy()
        if cluster_content=="performance":
            performance_cluster = clf.predict(cluster_dt)
        if cluster_content=="weapon":
            weapon_cluster = clf.predict(cluster_dt) 
            
    if cluster_method=="KMeans":
        model = KMeans(n_clusters=cluster_number, n_init=30)
        model.fit(cluster_dt)
        cluster_dt = cluster_dt.copy()
        if cluster_content=="performance":
            performance_cluster = model.predict(cluster_dt)
        if cluster_content=="weapon":
            weapon_cluster = model.predict(cluster_dt)        
            
    if cluster_method=="AA":
        cluster_matrix = cluster_dt.as_matrix()
        XC, S, C, SSE, varexpl = PCHA(cluster_matrix, noc=cluster_number, delta=0.1)
        XC_df=pd.DataFrame(XC)
        if cluster_content=="performance":
            performance_cluster = XC_df.idxmax(axis=1)
        if cluster_content=="weapon":
            weapon_cluster = XC_df.idxmax(axis=1)       
            
    return performance_cluster,weapon_cluster
        

In [15]:
cluster(dt=final_cluster_clean,cluster_content="performance",scale="MinMax", cluster_number=4, cluster_method="GMM")

(array([3, 3, 3, ..., 3, 3, 2]), [])